In [155]:
import os
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.utils import shuffle

In [156]:
articles_df = pd.read_csv("C:/Users/ojubh\/Desktop/SEMESTER 2/Deep Learning/Final_Project/data/articles.csv")

In [157]:
articles_df.columns

Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [158]:
sample_sub = pd.read_csv("C:/Users/ojubh\/Desktop/SEMESTER 2/Deep Learning/Final_Project/data/sample_submission.csv")

In [159]:
sample_sub.columns

Index(['customer_id', 'prediction'], dtype='object')

In [103]:
trans_train = pd.read_csv("C:/Users/ojubh\/Desktop/SEMESTER 2/Deep Learning/Final_Project/data/transactions_train.csv")

In [160]:
trans_train.columns # history puirchases

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id'], dtype='object')

In [152]:
image_embeddings = pd.read_csv("C:/Users/ojubh\/Desktop/SEMESTER 2/Deep Learning/Final_Project/data/visual_embeddings_final.csv")

In [154]:
image_embeddings.tail()

,Unnamed: 0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,article_id
104690,104690,-0.020810,-0.042550,-0.011611,0.012574,-0.045261,-0.044235,-0.027885,0.071938,0.037791,...,-0.025197,0.019250,0.007954,-0.057175,0.043820,-0.066785,0.122523,0.029554,-0.008207,711529001
104691,104691,0.012864,-0.076451,-0.002377,-0.033852,-0.023760,-0.018289,-0.023961,0.036726,0.014320,...,-0.037628,0.003001,0.032510,-0.023791,0.061463,-0.088391,0.081604,0.007389,-0.013648,685814005
104692,104692,0.010154,-0.068406,0.014912,-0.050652,-0.002357,-0.015711,-0.030078,0.039294,0.021177,...,0.011914,-0.010033,0.042382,-0.019953,0.014156,-0.079801,0.064390,0.010470,-0.002402,866529002
104693,104693,-0.002938,-0.020012,0.062998,0.040060,-0.015906,-0.010809,0.014671,0.056751,0.032178,...,-0.027348,-0.009032,0.065297,-0.040326,0.039135,-0.059344,0.020766,-0.060180,-0.011233,734461001
104694,104694,-0.065672,-0.048505,0.016058,0.033531,0.001518,0.028965,0.008971,0.092599,0.007461,...,-0.002169,0.011585,0.052929,-0.061341,0.078103,-0.049235,0.062557,-0.050831,0.001022,620037004


In [105]:
merged_customer_data = sample_sub.merge(trans_train, on="customer_id", how="inner")

In [106]:
merged_customer_data.drop(['t_dat', 'price', 'sales_channel_id'], inplace=True, axis=1)

In [107]:
customer_group = merged_customer_data.groupby("customer_id")

In [161]:
len(customer_group)

1362281

In [113]:
image_embeddings.drop("article_id", inplace=True, axis=1)
image_embeddings.drop("Unnamed: 0", axis=1, inplace=True)


In [118]:
x_train = image_embeddings.to_numpy()

In [119]:
neigh = NearestNeighbors(n_neighbors=12)
neigh.fit(x_train)

NearestNeighbors(n_neighbors=12)

In [ ]:
## 4 articles, sum up over all featurss/ len()
## numpy_arr = [...]

In [121]:
neigh.kneighbors(numpy_arr.reshape(1, -1))

(array([[0.        , 0.1990896 , 0.22100405, 0.24072063, 0.2425537 ,
         0.24804528, 0.248834  , 0.25008156, 0.25129139, 0.26048417,
         0.26257267, 0.26783031]]),
 array([[    0,  3126, 20782, 18722,  3480,  7166,   285,  6675, 15009,
         10820, 18750, 10864]], dtype=int64))